In [ ]:
import os
import sys
import pickle
import time

from copy import deepcopy

from cnp.experiment import WorkingDirectory

from cnp.cnp import StandardConvGNP
from cnp.lnp import StandardConvNP

from cnp.cov import (
    MeanFieldGaussianLayer,
    InnerprodGaussianLayer,
    KvvGaussianLayer,
    LogLogitCopulaLayer
)

import numpy as np
import matplotlib.pyplot as plt

import torch